In [1]:
import pandas as pd
import os

In [2]:
file_path = "c:/Users/User/Desktop/Project/data_project/car_scrape_pipeline/dags/etl/data/raw/list.csv"
columns_keyword = ["year_produce", "brand", "model","car_type", "gear_type", "engine_cap","mileage", "price"]

In [3]:
try:
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File {file_path} does not exist.")
    
    if file_path.endswith('.csv'):
        data = pd.read_csv(file_path, encoding='ISO-8859-1')
    elif file_path.endswith('.json'):
        data = pd.read_json(file_path)
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or JSON file.")

    df = data

except FileNotFoundError as e:
    print(e)
except Exception as e:
    print(f"An error occurred: {e}")

In [4]:
#remove duplicate
df_cleaned = df.drop_duplicates()

#Correcting invalid, inconsistent, or irrelevant data
for col in df_cleaned.columns:
    if 'make' in col.lower():
        df_cleaned = df_cleaned.rename(columns={"Make": "brand"})

for col in df_cleaned.columns:
    if 'car.type' in col.lower():
        df_cleaned["Car.Type"] = df_cleaned["Car.Type"].str.replace('Car', '')
        df_cleaned = df_cleaned.rename(columns={"Car.Type": "car_type"})

for col in df_cleaned.columns:
    if 'transm' in col.lower():   
        df_cleaned = df_cleaned.rename(columns={"Transm": "gear_type"})

for col in df_cleaned.columns:
    if 'engine.cap' in col.lower():
        df_cleaned["Engine.Cap"] = df_cleaned["Engine.Cap"].str.replace('-', '0')
        df_cleaned["Engine.Cap"] = df_cleaned["Engine.Cap"].str.replace('cc', '')
        df_cleaned["Engine.Cap"] = df_cleaned["Engine.Cap"].astype(float)
        df_cleaned = df_cleaned.rename(columns={"Engine.Cap": "engine_cap"})
        for index, row in df_cleaned.iterrows():
            value = float(row['engine_cap'])
            df_cleaned.at[index, 'engine_cap'] = [round(value / 1000, 1) if value > 100 else round(value)]


for col in df_cleaned.columns:
    if 'mileage' in col.lower():
        df_cleaned["Mileage"] = df_cleaned["Mileage"].fillna(0).astype(int)
        df_cleaned = df_cleaned.rename(columns={"Mileage": "mileage"})
        
for col in df_cleaned.columns:
    if 'price' in col.lower():   
        df_cleaned["Price"] = df_cleaned["Price"].astype(float).round(2)
        df_cleaned = df_cleaned.rename(columns={"Price": "old_price"})

df_cleaned.columns = df_cleaned.columns.str.lower()

#handle missing values (mean, median, mode, drop, estimate)
for index, row in df_cleaned[df_cleaned['engine_cap'] == 0].iterrows():
    model = row['model']
    year_produce = row['year']
    matching_row = df_cleaned[(df_cleaned['model'] == model) & (df_cleaned['year'] == year_produce) & (df_cleaned['engine_cap'] != 0)]
    if not matching_row.empty:
        df_cleaned.at[index, 'engine_cap'] = matching_row['engine_cap'].values[0]

for col in df_cleaned.columns:
    if 'year' in col.lower():
        df_cleaned = df_cleaned.rename(columns={"year": "year_produce"})


In [5]:
df = df_cleaned[["year_produce", "brand", "model","car_type", "gear_type", "engine_cap","mileage", "old_price"]]
    
base_name = os.path.basename(file_path)
name, ext = os.path.splitext(base_name)
new_file_name = f"old_{name}_clean{ext}"
df.to_csv(f'{new_file_name}')